In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Pipelines: reuse components

In [50]:
PROJECT_ID = "epishova-joonix-sandbox"

In [51]:
REGION = "us-central1"  # @param {type: "string"}

In [52]:
BUCKET_URI = "gs://epishova-team-pipelines"  # @param {type:"string"}

#### Service Account

**If you don't know your service account**, try to get your service account using `gcloud` command by executing the second cell below.

In [53]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [54]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    if IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

Service Account: 933318019720-compute@developer.gserviceaccount.com


In [55]:
PIPELINE_ROOT = "{}/pipeline_root/addition".format(BUCKET_URI)

In [56]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

In [57]:
from kfp import components

add = components.load_component_from_file(
    '/home/jupyter/mlops-demo/components/add_numbers/src/component_metadata/add.yaml')

In [58]:
from kfp import dsl

@dsl.pipeline(
    name="addition",
    pipeline_root=PIPELINE_ROOT,
)
def addition_pipeline(x: int, y: int) -> int:
    task1 = add(a=x, b=y)
    task2 = add(a=task1.output, b=x)
    return task2.output

In [59]:
compiler.Compiler().compile(
    pipeline_func=addition_pipeline, package_path='../pipelines/addition_pipeline.yaml'
)

In [61]:
DISPLAY_NAME = "addition"

job = aip.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="../pipelines/addition_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    parameter_values={'x':10, 'y':20},
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/933318019720/locations/us-central1/pipelineJobs/addition-20231008210632
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/933318019720/locations/us-central1/pipelineJobs/addition-20231008210632')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/addition-20231008210632?project=933318019720
PipelineJob projects/933318019720/locations/us-central1/pipelineJobs/addition-20231008210632 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/933318019720/locations/us-central1/pipelineJobs/addition-20231008210632 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/933318019720/locations/us-central1/pipelineJobs/addition-20231008210632 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/933318019720/locations/us-central1/pipelineJobs/addition-20231008210632 current state:
Pipel